1. control 변수 처리
2. 최적화 방향 처리
3. 중요도 처리


In [1]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

In [2]:
from deap import base, creator, tools
import numpy as np
import random

In [3]:
import faiss
from deap import algorithms

In [4]:
from operator import attrgetter


In [5]:
load_data_func = getattr(datasets, f'melb_data')
X_train, X_test, y_train, y_test = load_data_func('./data/melb_data_processed_04mean.csv')

load_data_loader_func = getattr(datasets, f'lightgbm_load_data')
train_loader, val_loader = load_data_loader_func(X_train, X_test, y_train, y_test)

train_func = getattr(surrogate, f'lightgbm_train')
model = train_func(train_loader, val_loader)


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:357: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 0.0711261	valid's rmse: 0.0879105
[200]	train's rmse: 0.0606034	valid's rmse: 0.0844337
[300]	train's rmse: 0.0540944	valid's rmse: 0.0837656
[400]	train's rmse: 0.0491065	valid's rmse: 0.0835851
[500]	train's rmse: 0.0450523	valid's rmse: 0.0832498
Early stopping, best iteration is:
[515]	train's rmse: 0.0444072	valid's rmse: 0.083166


In [6]:

predict_func = getattr(surrogate, f'lightgbm_predict')

In [7]:
gt_y = y_test[0]
gt_x = X_test[0]

In [8]:
vars = ['Suburb','Rooms','Type','Method','SellerG','Distance','Postcode','Bedroom2','Bathroom','Car','Landsize','YearBuilt','CouncilArea','Lattitude','Longtitude','Regionname','Propertycount','Address_bert_mean']

In [9]:
control = ['Landsize','Distance','Type']

In [10]:
optmize_dict = {'Landsize': 'minimize','Distance': 'maximize'}
importance = {'Distance': 2, 'Landsize': 1}

In [11]:
x_min,x_max = np.min(X_train, axis=0), np.max(X_train, axis=0)

In [12]:
# vars = ['Suburb','Rooms','Type','Price','Method','SellerG','Distance','Postcode','Bedroom2','Bathroom','Car','Landsize','YearBuilt','CouncilArea','Lattitude','Longtitude','Regionname','Propertycount','Address_bert_mean']

In [13]:
control_set = set(control)  
control_index = [i for i, v in enumerate(vars) if v in control_set] # var 중에 control

# control_importance_dict = {v: importance[v] for v in control if v in importance}   
control_index_to_pop_idx = {v: i+1 for i, v in enumerate(control_index)}



In [14]:
sorted_control_index_by_importance = sorted([i for i in control_index if vars[i] in importance.keys()], key=lambda x: importance[vars[x]]) # control중 importance 순서

In [15]:
sorted_pop_idx_by_importance = [control_index_to_pop_idx[i] for i in sorted_control_index_by_importance] 

In [18]:
sorted_optimize_dict_by_vars_idx = {vars[k]: optmize_dict[vars[k]] for k in [i for i in control_index if vars[i] in importance.keys()]}

In [57]:
def lexicographic_selection(population,k):

    # population.sort(key=lambda ind: tuple(ind.fitness.values[0],)+tuple(ind.fitness.values[i] for i in importance.values()), reverse=True)
    population.sort(
        key=lambda ind: tuple(ind.fitness.values[i] * ind.fitness.weights[i] for i in range(len(ind.fitness.values))),
        reverse=True
    )

    return population[:k] 

(-3.202095239120275e-07, 0.47762566363057474, -0.3309507382729003)

In [58]:
weights = tuple(1.0 if opt == 'maximize' else -1.0 for opt in sorted_optimize_dict_by_vars_idx.values())

In [59]:
# minmax_pop_idx = [control_index_to_pop_idx[i]-1 for i in control_index if vars[i] in importance.keys()]

In [60]:
# minmax_pop_idx

In [61]:
sorted_pop_idx_by_importance

[3, 2]

In [62]:
control_index

[2, 5, 10]

In [63]:
# creator.create('FitnessMax', base.Fitness, weights=(1.0,1.0,1.0))
creator.create('FitnessMax', base.Fitness, weights=(1.0,) + weights) # model pred + control optim 

creator.create('Individual', np.ndarray, fitness=creator.FitnessMax)
def generate_individual():
    return np.random.uniform(x_min[control_index], x_max[control_index])
toolbox = base.Toolbox()
toolbox.register('attr_float', generate_individual)
# min_max 차원이 8개이기에 n을 1로 설정 하면 8개의 변수를 가진 ind 생성!
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

/opt/conda/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/conda/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [64]:
def fitness(population):
    

    population = np.array(population)
    # print('population shape : ', population.shape)
    input_data = np.array(gt_x).reshape(1,-1).repeat(len(population), axis=0)
    input_data[:,control_index] = population
    y_pred = predict_func(model=model, X_test=input_data)

    fit_fun = []

    fit_fun.append(-(y_pred - gt_y)**2)
    # fit_fun = np.where(fit_fun > -0.01, 0, fit_fun)
    
    for i in sorted_pop_idx_by_importance:
        fit_fun.append(population[:,i-1:i])
    fit_fun = np.concatenate(fit_fun, axis=1)
    
    return fit_fun


In [65]:
attrgetter('fitness')

operator.attrgetter('fitness')

In [66]:
toolbox.register('evaluate', fitness)

In [67]:
ETA_CX = 5.0 # 초기에 작게(탐색), 점점 크게(exploitation)
toolbox.register('mate', tools.cxSimulatedBinary, eta=ETA_CX)

MUTPB = 0.2          # 돌연변이 적용 확률(전역)
INDPB = 0.1          # 각 변수별 변이 확률
sigma_list = [(ub - lb)/10.0 for (lb,ub) in zip(x_min, x_max)]  # 변수 범위에 따른 sigma 값 

toolbox.register('mutate', tools.mutGaussian, mu=[0.0]*(len(x_min)), sigma=sigma_list, indpb=INDPB)


In [68]:
toolbox.register('select', tools.selTournament)

In [69]:
population = toolbox.population(n=100)


In [70]:
population[0]

Individual([1.62235919, 0.82505372, 0.74260776])

In [71]:
cxpb = 0.5
mutpb = 0.5

In [72]:
def kmeans_clustering(population, k):
    n, d = population.shape
    kmeans = faiss.Kmeans(d, k, niter=20, verbose=False)
    population = population.astype('float32')
    kmeans.train(population)

    cluster_labels = kmeans.index.search(population, 1)[1].flatten()  
    centroids = kmeans.centroids
    
    return cluster_labels, centroids


In [73]:
# def k_means_selection(population, k):
#     flag = 0

#     cluster_labels, centroids = kmeans_clustering(np.array(population),k=k)
#     res = []
#     for i in range(k):
#         cluster_idx = np.where(cluster_labels == i)[0]
#         cluster_population = [population[j] for j in cluster_idx]
        
#         # 홀수일 때 population 수 줄어듦 방지 
#         if len(cluster_population)%2 == 0:
#             res.extend(lexicographic_selection(cluster_population, k=len(cluster_population)//2,sorted_control_importance_rank = sorted_control_importance_rank))
#         else:
#             if flag == 0:
#                 res.extend(lexicographic_selection(cluster_population, k=len(cluster_population)//2+1,sorted_control_importance_rank = sorted_control_importance_rank))
#                 flag = 1
#             else:
#                 res.extend(lexicographic_selection(cluster_population, k=len(cluster_population)//2,sorted_control_importance_rank = sorted_control_importance_rank))
#                 flag = 0
#     return res


In [74]:
def k_means_selection(population, k):
    """
    Args:
        population (list)
        k (int): K-means에서 나눌 클러스터 개수.

    Returns:
        list: 선택된 ind 리스트.
    """
    # print('population len : ',len(population))
    adjustment_flag = False  # 클러스터 크기가 홀수일 때 조절하는 플래그
    population_array = np.array(population)
    
    cluster_labels, _ = kmeans_clustering(population_array, k=k)
    
    selected = []
    
    for cluster_id in range(k):
        cluster_indices = np.where(cluster_labels == cluster_id)[0]
        cluster_population = [population[idx] for idx in cluster_indices]
        cluster_size = len(cluster_population)
        
        # 홀수일 떄 개체수 줄어듦 방지 
        selection_size = cluster_size // 2
        if cluster_size % 2 == 1 and not adjustment_flag:
            selection_size += 1
            adjustment_flag = True
        elif cluster_size % 2 == 1 and adjustment_flag:
            adjustment_flag = False
        
        selected.extend(
            lexicographic_selection(
                cluster_population, 
                k=selection_size, 
            )
        )
    # print(len(selected))
    return selected


In [75]:
# %%capture
for i in range(100):
    offspring = algorithms.varAnd(population, toolbox, cxpb, mutpb)
    population = offspring+population
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitness_scores = toolbox.evaluate(invalid_ind)
    for ind, fit in zip(invalid_ind, fitness_scores):
        # print(fit)
        ind.fitness.values = tuple(fit,)
        # print(ind.fitness.values)
    population = k_means_selection(population, k=len(population)//10)
    # print([i.fitness.values for i in population[:3]])

WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points
WARNING clustering 200 points to 20 centroids: please provide at least 780 training points

In [76]:
ind.fitness.values

(-3.202095239120275e-07, 0.47762566363057474, 0.3309507382729003)

In [77]:
ind

Individual([-8.8368163 ,  0.33095074,  0.47762566])

In [80]:
tuple(ind.fitness.values[i] * ind.fitness.weights[i] for i in range(len(ind.fitness.values)))

(-3.202095239120275e-07, 0.47762566363057474, -0.3309507382729003)